# basicsynbio Addgene submission collection

This Notebook contains code to aid the submission of basicsynbio linkers to addgene.

## Notes on data preperation

- [x] The p15A plasmid supplied by Marko `BS_x6x` was corrected given it has a "t" insertion.
- [x] The Benchling feature library (BASIC_SEVA_benchling_misc_features_library.csv) was updated commit: `a2fb18c`.

## Aims and objectives for cell/s below


- [x] Import the following objects:
  - [x] pickled BasicLinker `SEVA-BB1`
  - [x] Selected Oris and AbR markers specified in #130.
  - [x] mScarlet counter selection casettes for associated Oris.
- [x] Generate a dictionary mapping between Oris and counter-selection casettes.
- [ ] Make assemblies and export the following data:
  - [x] Genbank file containing all correctly assembled backbones.
  - [x] Manual and automated build instructions.

In [1]:
import basicsynbio as bsb
from basicsynbio.utils import MARKER_DICT, ORI_DICT
from pathlib import Path
import pickle

In [2]:
path_to_seqs = Path.cwd().parents[0] / "sequences"
path_to_initial_mods = path_to_seqs / "genbank_files" / "BASIC_SEVA_collection" / "initial_mods"
# Import objects
with open(path_to_seqs / "alternative_formats" / "pickles" / "SEVA-BB1", "rb") as file:
    bb_linker = pickle.load(file)
abr_markers = list(
    bsb.import_parts(path_to_initial_mods / "AbR_markers.gb",
    "genbank"
))
oris = list(
    bsb.import_parts(path_to_initial_mods / "oris.gb",
    "genbank"
))
cs_cassettes = list(
    bsb.import_parts(path_to_initial_mods / "mScarlet_cs_cassettes.gb",
    "genbank"
))
# Map oris with counter selection cassettes
ori_cs_cassette_mapping = {
    "BS_x5x": "B407_J23119-RBS34-mScarl",
    "BS_x6x": "B407_J23119-RBS34-mScarl",
    "BS_x7x": "B408_J23119-RBS-A12-mSc",
    "BS_x7x_TS_CASc": "B408_J23119-RBS-A12-mSc",
    "BS_x8x": "B405_J23106-RBS34-mScarl",
    "BS_x9x": "B407_J23119-RBS34-mScarl",
}
ori_cs_cassette_mapping = {
    key: cs_cassettes[[cs_cassette.id for cs_cassette in cs_cassettes].index(value)] for key, value in ori_cs_cassette_mapping.items()
}
# Make assemblies and export data
linkers = bsb.BASIC_BIOLEGIO_LINKERS["v0.1"]
assemblies = []
for abr_marker in abr_markers:
    for ori in oris:    
        assembly = bsb.BasicAssembly(
            "foobar",
            linkers["LMP"],
            abr_marker,
            bb_linker,
            ori,
            linkers["LMS"],
            ori_cs_cassette_mapping[ori.id],
        )
        if ori.id == "BS_x7x_TS_CASc":
            assembly.id = f"BASIC_SEVA_{abr_marker.id[-3]}7_TS.1"
            assembly.description = f"BASIC SEVA vector containing {MARKER_DICT[abr_marker.id[-3]]} resistance marker and temperature sensitive pSC101 origin of replication."
        else:
            assembly.id = f"BASIC_SEVA_{abr_marker.id[-3] + ori.id[-2]}.10"
            assembly.description = f"BASIC SEVA vector containing {MARKER_DICT[abr_marker.id[-3]]} resistance marker and {ORI_DICT[ori.id[-2]]} origin of replication."
        assemblies.append(assembly)
build = bsb.BasicBuild(*assemblies)
bsb.export_sequences_to_file(
    (assembly.return_part(description=assembly.description, name=assembly.id) for assembly in build.basic_assemblies),
    Path.cwd().parents[0] / "basicsynbio" / "parts_linkers" / "BASIC_SEVA_collection_v10.gb"
)
bsb.export_echo_assembly(
    build,
    Path.cwd().parents[0] / "csv_xlsx_files" / "BASIC_SEVA_collection_v10_echo_scripts.zip",
    assemblies_per_clip=20,
    alternate_well=True,
)
bsb.pdf_instructions(
    build,
    "/home/hainesm6/github_repos/LondonBiofoundry/basicsynbio/pdfs/BASIC_SEVA_collection_v10_manual.pdf",
    assemblies_per_clip=20,
)



'/home/hainesm6/github_repos/LondonBiofoundry/basicsynbio/pdfs/BASIC_SEVA_collection_v10_manual.pdf'

In [3]:
print([abr_marker.id for abr_marker in abr_markers])

['BS_1xx', 'BS_2xx', 'BS_3xx', 'BS_4xx', 'BS_5xx', 'BS_6xx']
